In [12]:
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle
from six.moves import range
from datetime import datetime
tf.test.is_gpu_available()

True

In [16]:
# unpickle

pickle_file = 'SVHN.pickle'

with open(pickle_file, 'rb') as f:
    save = pickle.load(f)
    train_dataset = save['train_dataset']
    train_labels = save['train_labels']
    valid_dataset = save['valid_dataset']
    valid_labels = save['valid_labels']
    test_dataset = save['test_dataset']
    test_labels = save['test_labels']
    del save  # hint to help gc free up memory
    
    
pickle_file = 'SVHN_extra.pickle'

with open(pickle_file, 'rb') as f:
    save = pickle.load(f)
    train2_dataset = save['train2_dataset']
    train2_labels = save['train2_labels']
    del save  # hint to help gc free up memory

pickle_file = 'SVHN_extra2.pickle'

with open(pickle_file, 'rb') as f:
    save = pickle.load(f)
    train3_dataset = save['train3_dataset']
    train3_labels = save['train3_labels']
    del save  # hint to help gc free up memory

print('Training set', train_dataset.shape, train_labels.shape)
print('Training set extra 1', train2_dataset.shape, train2_labels.shape)
print('Training set extra 2', train3_dataset.shape, train3_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 32, 32) (200000,)
Training set extra 1 (200000, 32, 32) (200000,)
Training set extra 2 (143949, 32, 32) (143949,)
Validation set (60439, 32, 32) (60439,)
Test set (26032, 32, 32) (26032,)


In [17]:
# concatenate training sets
train_dataset = np.concatenate((train_dataset, train2_dataset, train3_dataset), axis=0)
train_labels = np.concatenate((train_labels, train2_labels, train3_labels), axis=0)
del train2_dataset, train3_dataset, train2_labels, train3_labels
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (543949, 32, 32) (543949,)
Validation set (60439, 32, 32) (60439,)
Test set (26032, 32, 32) (26032,)


In [23]:
# reformat dataset for training

image_size = 32
num_labels = 10
num_channels = 1 # grayscale

def reformat(dataset, labels):
    dataset = dataset.reshape((-1, image_size, image_size, num_channels)).astype(np.float32)
    labels = labels.astype(np.int32)
    return dataset, labels

train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)

print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (543949, 32, 32, 1) (543949,)
Validation set (60439, 32, 32, 1) (60439,)
Test set (26032, 32, 32, 1) (26032,)


In [24]:
# helper methods for creating CNN
def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)

def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

def conv2d_same(x, W):
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

def conv2d_valid(x, W):
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='VALID')

def max_pool_2x2(x):
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1],strides=[1, 2, 2, 1], padding='SAME')

def accuracy(predictions, labels):
    return (100.0 * np.sum(np.argmax(predictions, 1) == labels) / predictions.shape[0])

In [25]:
# define parameters for model

batch_size = 64
patch_size = 5
depth1 = 16
depth2 = 32
depth3 = 64
num_hidden1 = 64
num_hidden2 = 16
shape=[batch_size, image_size, image_size, num_channels]

graph = tf.Graph()

with graph.as_default():

    # Input parameters
    tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size, image_size, num_channels))
    tf_train_labels = tf.placeholder(tf.int64, shape=(batch_size))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
  

    # Weight and bias variables
    W_C1 = weight_variable([patch_size, patch_size, num_channels, depth1])
    b_C1 = tf.Variable(tf.constant(1.0, shape=[depth1]), name='B1')
    
    W_C2 = weight_variable([patch_size, patch_size, depth1, depth2])
    b_C2 = tf.Variable(tf.constant(1.0, shape=[depth2]), name='B2')
    
    W_C3 = weight_variable([patch_size, patch_size, depth2, num_hidden1])
    b_C3 = tf.Variable(tf.constant(1.0, shape=[num_hidden1]), name='B3')
    
    W_C4 = weight_variable([num_hidden1, num_hidden2])
    b_C4 = tf.Variable(tf.constant(1.0, shape=[num_hidden2]), name='B4')
    
    W_C5 = weight_variable([num_hidden2, num_labels])
    b_C5 = tf.Variable(tf.constant(1.0, shape=[num_labels]), name='B5')
  
    # neural network model
    def model(data, keep_prob, shape):
        # C1: convolutional layer, batch_size x 28 x 28 x 16, convolution size: 5 x 5 x 1 x 16
        conv =conv2d_valid(data, W_C1)
        hconv = tf.nn.relu(conv + b_C1)
        
        #Local response normalization
        lrn = tf.nn.local_response_normalization(hconv)
        
        # S2: sub-sampling layer, batch_size x 14 x 14 x 16
        maxpool = max_pool_2x2(lrn)
        
        # C3: convolutional layer, batch_size x 10 x 10 x 32, convolution size: 5 x 5 x 16 x 32
        conv =conv2d_valid(maxpool, W_C2)
        hconv = tf.nn.relu(conv + b_C2)
        
        #Local response normalization
        lrn = tf.nn.local_response_normalization(hconv)
        
        # S4: sub-sampling layer, batch_size x 5 x 5 x 32
        maxpool = max_pool_2x2(lrn)
        
        # C5: convolutional layer, batch_size x 1 x 1 x 64, convolution size: 5 x 5 x 32 x 64
        conv =conv2d_valid(maxpool, W_C3)
        hconv = tf.nn.relu(conv + b_C3)
        
        # Dropout
        dropout = tf.nn.dropout(hconv, keep_prob)
        
        # Reshaping tensor for fully connected layers
        shape = dropout.get_shape().as_list()
        flattened = tf.reshape(dropout, [shape[0], shape[1] * shape[2] * shape[3]])
        
        # F6: fully-connected layer, shape: 64 x 16
        FC = tf.nn.relu(tf.matmul(flattened, W_C4) + b_C4)
        
        # Output layer, weight size: 16 x 10
        return tf.matmul(FC, W_C5) + b_C5
    
    # defining logits and loss function
    logits = model(tf_train_dataset, 0.9, shape)
    loss = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(logits, tf_train_labels))
    
    # Defining learning rate and optimizer
    global_step = tf.Variable(0)
    learning_rate = 0.1
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
  
    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(model(tf_train_dataset, 1.0, shape))
    valid_prediction = tf.nn.softmax(model(tf_valid_dataset, 1.0, shape))
    test_prediction = tf.nn.softmax(model(tf_test_dataset, 1.0, shape))
    
    # Save model
    saver = tf.train.Saver()

In [26]:
# starting session and run computation

num_steps = 200001
print('Training start time: ', str(datetime.now()))
with tf.Session(graph=graph,config=tf.ConfigProto(log_device_placement=True)) as session:
    tf.initialize_all_variables().run()
    print('Initialized')
    for step in range(num_steps):
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
        batch_labels = train_labels[offset:(offset + batch_size)]
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 500 == 0): 
            print('Minibatch loss at step %d: %f' % (step, l))
            print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
            print('Validation accuracy: %.1f%%' % accuracy(valid_prediction.eval(), valid_labels))
    print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))
    save_path = saver.save(session, "SVHN_single.ckpt")
    print('Training end time: ', str(datetime.now()))
    print("Model saved in file: %s" % save_path)

Training start time:  2016-12-01 19:23:48.771280
Initialized
Minibatch loss at step 0: 2.531307
Minibatch accuracy: 10.9%
Validation accuracy: 14.2%
Minibatch loss at step 500: 0.645538
Minibatch accuracy: 85.9%
Validation accuracy: 82.0%
Minibatch loss at step 1000: 0.426315
Minibatch accuracy: 93.8%
Validation accuracy: 88.8%
Minibatch loss at step 1500: 0.321635
Minibatch accuracy: 87.5%
Validation accuracy: 90.8%
Minibatch loss at step 2000: 0.430683
Minibatch accuracy: 87.5%
Validation accuracy: 91.5%
Minibatch loss at step 2500: 0.308262
Minibatch accuracy: 92.2%
Validation accuracy: 91.6%
Minibatch loss at step 3000: 0.390852
Minibatch accuracy: 90.6%
Validation accuracy: 92.8%
Minibatch loss at step 3500: 0.202205
Minibatch accuracy: 93.8%
Validation accuracy: 92.8%
Minibatch loss at step 4000: 0.155466
Minibatch accuracy: 95.3%
Validation accuracy: 93.0%
Minibatch loss at step 4500: 0.292037
Minibatch accuracy: 90.6%
Validation accuracy: 93.4%
Minibatch loss at step 5000: 0.27